In [1]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

2023-01-06 20:13:56.667034: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
symbol = "BTCUSDT"
PERIOD = "15m"

In [14]:
!jupyter nbconvert --to python ../collect/get_newest_price.ipynb
!python ../collect/get_newest_price.py

[NbConvertApp] Converting notebook ../collect/get_newest_price.ipynb to python
[NbConvertApp] Writing 1547 bytes to ../collect/get_newest_price.py


# Origin data

In [15]:
pd_df = pd.read_csv(f"../../datastore/price/{symbol}_{PERIOD}.csv")
pd_df

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1517500800000,9326.23,9340.00,9195.89,9217.00,487.476964,1517501699999,4.506285e+06,4546,171.643712,1.586177e+06
1,1517501700000,9208.09,9275.00,9130.00,9145.00,652.169234,1517502599999,6.004393e+06,6930,334.699485,3.083885e+06
2,1517502600000,9144.98,9257.40,9020.00,9184.99,1028.146331,1517503499999,9.361539e+06,8632,376.048437,3.429102e+06
3,1517503500000,9184.98,9270.00,9011.51,9066.66,770.943401,1517504399999,7.037635e+06,6722,356.335133,3.256645e+06
4,1517504400000,9058.78,9156.98,9040.00,9106.22,490.060336,1517505299999,4.458077e+06,4557,244.406918,2.225572e+06
...,...,...,...,...,...,...,...,...,...,...,...
172265,1673007300000,16737.15,16738.28,16719.09,16721.17,1919.983110,1673008199999,3.212120e+07,52028,945.981120,1.582643e+07
172266,1673008200000,16721.17,16738.14,16717.66,16733.72,1039.658290,1673009099999,1.739167e+07,28824,559.994890,9.367704e+06
172267,1673009100000,16733.79,16744.95,16727.35,16731.02,1744.702660,1673009999999,2.919968e+07,53661,863.686280,1.445509e+07
172268,1673010000000,16730.65,16731.84,16716.80,16720.92,404.021890,1673010899999,6.756612e+06,10933,166.867140,2.790580e+06


### Format open price and time

In [16]:
pd_df["Open price"] = pd_df["Open price"].apply(lambda price: float(price))
pd_df = pd_df.set_index("Kline open time").sort_index() 
pd_df.index = pd.to_datetime(pd_df.index, unit='ms') + pd.Timedelta('07:00:00')
pd_df

,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Kline open time,,,,,,,,,,
2018-02-01 23:00:00,9326.23,9340.00,9195.89,9217.00,487.476964,1517501699999,4.506285e+06,4546,171.643712,1.586177e+06
2018-02-01 23:15:00,9208.09,9275.00,9130.00,9145.00,652.169234,1517502599999,6.004393e+06,6930,334.699485,3.083885e+06
2018-02-01 23:30:00,9144.98,9257.40,9020.00,9184.99,1028.146331,1517503499999,9.361539e+06,8632,376.048437,3.429102e+06
2018-02-01 23:45:00,9184.98,9270.00,9011.51,9066.66,770.943401,1517504399999,7.037635e+06,6722,356.335133,3.256645e+06
2018-02-02 00:00:00,9058.78,9156.98,9040.00,9106.22,490.060336,1517505299999,4.458077e+06,4557,244.406918,2.225572e+06
...,...,...,...,...,...,...,...,...,...,...
2023-01-06 19:15:00,16737.15,16738.28,16719.09,16721.17,1919.983110,1673008199999,3.212120e+07,52028,945.981120,1.582643e+07
2023-01-06 19:30:00,16721.17,16738.14,16717.66,16733.72,1039.658290,1673009099999,1.739167e+07,28824,559.994890,9.367704e+06
2023-01-06 19:45:00,16733.79,16744.95,16727.35,16731.02,1744.702660,1673009999999,2.919968e+07,53661,863.686280,1.445509e+07


In [1]:
# fig = go.Figure(data=[go.Candlestick(x=pd_df.index,
#                 open=pd_df['Open price'],
#                 high=pd_df['High price'],
#                 low=pd_df['Low price'],
#                 close=pd_df['Close price'])])
# # set new height and width
# fig.update_layout(
#     height=800,
#     width=1000,
#     title_text="BTC/USDT price",
#     yaxis_title="Price (BTC/USDT)",
#     xaxis_title="Date",
#     xaxis_rangeslider_visible=True
# )

# fig.show()

# Prepare dataset

### Price column

In [97]:
dataset = pd_df.filter(["Open price"]).values

### Splitting ratio

In [98]:
TRAIN_DATA_LENGTH = int(len(dataset) * 0.8)

### Scale data to [0, 1]

In [99]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.0934913 ],
       [0.0880261 ],
       [0.09310943],
       ...,
       [0.2093466 ],
       [0.21206311],
       [0.2117753 ]])

### Split training set

In [100]:
train_data = scaled_data[:TRAIN_DATA_LENGTH]

X_train = []
Y_train = []
for i in range(60,len(train_data)):
    X_train.append(train_data[i-60:i,:])
    Y_train.append(train_data[i,:])

X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

### Split test set

In [101]:
test_data = scaled_data[TRAIN_DATA_LENGTH-60:]

X_test = []
Y_test = []
for i in range(60,len(test_data)):
    X_test.append(test_data[i-60:i,:])
    Y_test.append(test_data[i,:])

X_test, Y_test = np.array(X_test), np.array(Y_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Training

## Long-short term memory model

In [102]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(200, return_sequences=False))
model.add(Dense(100))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, Y_train, batch_size=1, epochs=2)

Epoch 1/2
1380/1380 [==============================] - 29s 20ms/step - loss: 0.0038
Epoch 2/2
1380/1380 [==============================] - 25s 18ms/step - loss: 0.0014


### Predict and transform [0, 1] to the original scale

In [103]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

12/12 [==============================] - 1s 26ms/step


### Visualize the prediction

In [104]:
data = pd_df.filter(["Open price"])
train = data[:TRAIN_DATA_LENGTH]
valid = data[TRAIN_DATA_LENGTH:].reset_index()
valid["Predict"] = predictions
valid = valid.set_index("Kline open time")
valid["Predict"]= valid["Predict"].apply(lambda price: float(price))

lstm_df = pd.concat([train, valid], axis=0)

In [105]:
fig = px.line(lstm_df, title="BTC/USDT price" , width=1000, height=800)
fig.show()


## Autoregressive integrated moving average model

In [106]:
from statsmodels.tsa.arima.model import ARIMA

### Scale data to [0, 1] and split train-test set

In [107]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)
train, test = scaled_data[:TRAIN_DATA_LENGTH], scaled_data[TRAIN_DATA_LENGTH:]

### Train model

In [108]:
history = train.copy().tolist()

predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)
    actual_price = test[t]
    history.append(actual_price)

/Users/na/opt/miniconda3/envs/NaDS/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



### Visualize the prediction

In [109]:
train_df, test_df = data[:TRAIN_DATA_LENGTH], data[TRAIN_DATA_LENGTH:]
test_df["Predict"] = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).reshape(-1)
arima_df = pd.concat([train_df, test_df], axis=0)


fig = px.line(arima_df, title="BTC/USDT price" , width=1000, height=800)
fig.show()               

/var/folders/p7/08wxvf195r35gt2pw893ynsw0000gn/T/ipykernel_2422/3839462130.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Model selection with cross validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from estimator import ArimaEstimator, LSTMEstimator, SarimaxEstimator, AutoRegEstimator, ExponentialSmoothingEstimator, HoltEstimator

# cross vadiation strategy
tscv = TimeSeriesSplit(n_splits=5)

# mean squared error metric
metric = make_scorer(mean_squared_error)

# models
estimators = [
    ArimaEstimator(), 
    SarimaxEstimator(), 
    AutoRegEstimator(),
    ExponentialSmoothingEstimator(), 
    HoltEstimator(),
    LSTMEstimator(), 
]

scores_dict = {}

for estimator in estimators:
    scores = cross_val_score(estimator, X_train, Y_train, scoring=metric, cv=tscv, n_jobs=-1)
    print(estimator.__class__.__name__, scores.mean())
    scores_dict[estimator.__class__.__name__] = scores
    
scores_df = pd.DataFrame(scores_dict)
scores_df.index = scores_df.index.map(lambda x: f'fold_{x+1}')
scores_df.loc['mean'] = scores_df.mean()
scores_df.to_csv('../../datastore/scores.csv')

In [111]:
scores_df

,ArimaEstimator,SarimaxEstimator,AutoRegEstimator,ExponentialSmoothingEstimator,HoltEstimator,LSTMEstimator
fold_1,0.000027,0.000032,0.000022,0.000020,0.000022,0.000558
fold_2,0.000027,0.000050,0.000025,0.000024,0.000026,0.000038
fold_3,0.000036,0.000052,0.000034,0.000031,0.000033,0.000064
fold_4,0.000840,0.001099,0.000818,0.000777,0.000807,0.004770
fold_5,0.000769,0.001182,0.000754,0.000696,0.000722,0.003198
mean,0.000340,0.000483,0.000331,0.000310,0.000322,0.001726


In [112]:
# find column with lowest mean score
best_estimator = scores_df.loc['mean'].idxmin()
best_estimator

'ExponentialSmoothingEstimator'

In [ ]:
%%capture
from estimator import ExponentialSmoothingEstimator as Estimator
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

tscv = TimeSeriesSplit()

metric = make_scorer(mean_squared_error, greater_is_better=False)

param_grid = {
    'trend': ["add", "mul", "additive", "multiplicative", None],
    'damped_trend': [True, False],
    'seasonal': ["add", "mul", "additive", "multiplicative", None],
    'seasonal_periods': [7, 30, 365],
}

estimator = Estimator()
estimator.get_params().keys()
grid_search = GridSearchCV(estimator, param_grid, scoring=metric, cv=tscv, n_jobs=-1, verbose=0)
grid_search.fit(X_train, Y_train)

In [114]:
grid_search.best_estimator_

ExponentialSmoothingEstimator(seasonal_periods=7)

# Params selection with grid search

In [115]:
from statsmodels.tsa.api import ExponentialSmoothing

data = pd_df.filter(["Open price"])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)
train, test = scaled_data[:TRAIN_DATA_LENGTH], scaled_data[TRAIN_DATA_LENGTH:]
history = train.copy().tolist()

predictions = list()
for t in range(len(test)):
    model = ExponentialSmoothing(history, seasonal_periods=7)
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)
    actual_price = test[t]
    history.append(actual_price)

In [116]:

train_df, test_df = data[:TRAIN_DATA_LENGTH], data[TRAIN_DATA_LENGTH:]
test_df["Predict"] = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).reshape(-1)


# concate train_df and test_df to get the full data set
full_df = pd.concat([train_df, test_df], axis=0)


fig = px.line(full_df, height=800, width=1000)
fig.show()

/var/folders/p7/08wxvf195r35gt2pw893ynsw0000gn/T/ipykernel_2422/704318073.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [117]:
model = ExponentialSmoothing(data["Open price"].values, seasonal_periods=7)
model_fit = model.fit()
output = model_fit.forecast()
pred_price = output[0]

In [118]:
pred_price

16832.28719995541

In [119]:
data

,Open price
Kline open time,
2018-02-02,9224.52
2018-02-03,8873.03
2018-02-04,9199.96
2018-02-05,8179.99
2018-02-06,6939.63
...,...
2023-01-02,16617.17
2023-01-03,16672.78
2023-01-04,16675.65
